In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
import warnings 
warnings.filterwarnings('ignore')

In [2]:
from IPython.display import display
import ipywidgets as widgets
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
colnames=['movieId', 'title', 'genres']
movies = pd.read_csv("data//ml-1m//movies.dat",sep='::',encoding ='latin-1',header = None,names=colnames)
print('movies.shape : ',movies.shape)
movies.head()

movies.shape :  (3883, 3)


,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [59]:
movies.tail(20)

,movieId,title,genres
3863,3933,"Killer Shrews, The (1959)",Horror|Sci-Fi
3864,3934,Kronos (1957),Sci-Fi
3865,3935,Kronos (1973),Horror
3866,3936,"Phantom of the Opera, The (1943)",Drama|Thriller
3867,3937,Runaway (1984),Sci-Fi|Thriller
3868,3938,"Slumber Party Massacre, The (1982)",Horror
3869,3939,"Slumber Party Massacre II, The (1987)",Horror
3870,3940,"Slumber Party Massacre III, The (1990)",Horror
3871,3941,Sorority House Massacre (1986),Horror
3872,3942,Sorority House Massacre II (1990),Horror


In [38]:
colnames=['userId', 'movieId', 'rating', 'timestamp']
ratings = pd.read_csv("data//ml-1m//ratings.dat",sep='::',encoding ='latin-1',header = None,names=colnames,usecols=colnames[:-1])
print('ratings.shape : ',ratings.shape)
ratings.head()

ratings.shape :  (1000209, 3)


,userId,movieId,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [22]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 3 columns):
 #   Column   Non-Null Count    Dtype
---  ------   --------------    -----
 0   userId   1000209 non-null  int64
 1   movieId  1000209 non-null  int64
 2   rating   1000209 non-null  int64
dtypes: int64(3)
memory usage: 22.9 MB


In [8]:
rat_count_mean = pd.DataFrame(ratings.groupby('movieId')['rating'].agg(['mean','count']).sort_values(by = 'count',ascending=False))

In [9]:
rat_count_mean

,mean,count
movieId,,
2858,4.317386,3428
260,4.453694,2991
1196,4.292977,2990
1210,4.022893,2883
480,3.763847,2672
...,...,...
3237,1.000000,1
763,3.000000,1
624,4.000000,1


In [10]:
rat_count_mean['count'].describe()

count    3706.000000
mean      269.889099
std       384.047838
min         1.000000
25%        33.000000
50%       123.500000
75%       350.000000
max      3428.000000
Name: count, dtype: float64

In [11]:
rat_count_mean_30 = rat_count_mean[rat_count_mean['count']>200]

In [12]:
movies.shape

(3883, 3)

In [13]:
rat_count_mean_30.shape

(1420, 2)

In [14]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   userId     1000209 non-null  int64
 1   movieId    1000209 non-null  int64
 2   rating     1000209 non-null  int64
 3   timestamp  1000209 non-null  int64
dtypes: int64(4)
memory usage: 30.5 MB


In [23]:
ratings[ratings.movieId.isin(rat_count_mean_30.index)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 854530 entries, 0 to 1000208
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   userId   854530 non-null  int64
 1   movieId  854530 non-null  int64
 2   rating   854530 non-null  int64
dtypes: int64(3)
memory usage: 26.1 MB


In [24]:
rat_count_mean_30_3_5 = rat_count_mean_30[rat_count_mean_30['mean']>3.5]

In [25]:
rat_count_mean_30_3_5.shape

(781, 2)

In [48]:
ratings = ratings[(ratings.rating>4)]#.info()

In [49]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226310 entries, 0 to 1000206
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   userId   226310 non-null  int64
 1   movieId  226310 non-null  int64
 2   rating   226310 non-null  int64
dtypes: int64(3)
memory usage: 6.9 MB


In [29]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  3883 non-null   int64 
 1   title    3883 non-null   object
 2   genres   3883 non-null   object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB


In [7]:
movies[movies.movieId==5349]

,movieId,title,genres
856,5349,Spider-Man (2002),Action|Adventure|Sci-Fi|Thriller


In [6]:
data_folder = 'C:\\Users\\user\\Solo Dolo\\Recommendation System\\Movie\\'

In [7]:
movies = pd.read_csv(data_folder+'artifacts//movies.csv')
print('movies.shape : ',movies.shape)
movies.head()

movies.shape :  (1223, 3)


,movieId,title,genres
0,1,Toy Story 1995,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji 1995,Adventure|Children|Fantasy
2,3,Grumpier Old Men 1995,Comedy|Romance
3,5,Father of the Bride Part II 1995,Comedy
4,6,Heat 1995,Action|Crime|Thriller


In [9]:
a = movies.loc[(movies.title =='Jumanji 1995'),'movieId'].values[0]

In [27]:
a#.values[0]

2

In [8]:
ratings = pd.read_csv(data_folder+'artifacts//ratings.csv')
print('ratings.shape : ',ratings.shape)
ratings.head()

ratings.shape :  (3122524, 3)


,userId,movieId,rating
0,2,110,5.0
1,2,151,4.5
2,2,260,5.0
3,2,318,5.0
4,2,333,5.0


In [12]:
def find_similar_movies(movie_id):
    #get similar users that rated the movie above 4
    similar_users = ratings[
        (ratings['movieId'] == movie_id) & (ratings['rating']>4)
    ]["userId"].unique()
    
    #find other movies they similars users rated above 4
    similar_users_recs = ratings[
        (ratings["userId"].isin(similar_users)) & (ratings['rating']>4)
    ]['movieId']
    
    #find the percentage of similar_users who liked the  movies particular to that niche i.e movies that a greater than 10% of the users similar to us liked
    similar_users_recs = similar_users_recs.value_counts()/len(similar_users)
    similar_users_recs = similar_users_recs[similar_users_recs > .1]
    
    #find how much all_users liked these movies in the niche >10% i.e (including users outside that niche)
    all_users = ratings[
        (ratings['movieId'].isin(similar_users_recs.index)) & (ratings['rating']>4)
    ]
    
    #find the percentage of all users who liked each movie recommendend
    all_users_recs = all_users['movieId'].value_counts()/len(all_users['userId'].unique())
    
    #combine the percentage of simlar_users_rec and all_users_recs
    rec_percentage = pd.concat([similar_users_recs,all_users_recs],axis=1)
    rec_percentage.columns = ['similar','all']
    
    #we want movies that have big bifference between the two numbers
    rec_percentage['score'] = rec_percentage['similar']/rec_percentage['all'] 
    #sort_values
    rec_percentage = rec_percentage.sort_values('score',ascending=False)
    #get top 10  recommendations
    rec_movies =  rec_percentage.head(10).merge(movies,left_index=True , right_on='movieId')[['title','genres','score','movieId']]

    return rec_movies.title.to_dict()

find_similar_movies(movie_id=5349)

{856: 'SpiderMan 2002',
 952: 'SpiderMan 2 2004',
 896: 'X2 XMen United 2003',
 1015: 'XMen The Last Stand 2006',
 759: 'XMen 2000',
 1046: 'Transformers 2007',
 858: 'Star Wars Episode II  Attack of the Clones 2002',
 1109: 'Iron Man 2 2010',
 866: 'Signs 2002',
 954: 'I Robot 2004'}

In [13]:
import requests
url = 'http://localhost:9696/recommend'

In [26]:
response = requests.post(url, json = {'title':'Iron Man 2 2010'})

In [27]:
response

<Response [200]>

In [28]:
result = response.json()
result

{'1069': 'Incredible Hulk The 2008',
 '1072': 'Hancock 2008',
 '1089': 'XMen Origins Wolverine 2009',
 '1109': 'Iron Man 2 2010',
 '1127': 'Thor 2011',
 '1133': 'Captain America The First Avenger 2011',
 '1153': 'Amazing SpiderMan The 2012',
 '1165': 'Iron Man 3 2013',
 '1174': 'Thor The Dark World 2013',
 '1203': 'Avengers Age of Ultron 2015'}